In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import frame
import block_DCT
import YCoCg as YUV
import cv2

## Testing `block_DCT.block_analyze()` and `block_DCT.block_synthesize()`

In [ ]:
#a = np.random.randint(low=0, high=100, size=(4,4,3))
a = np.full(shape=(4,4,3), fill_value=10, dtype=np.int16)

In [ ]:
print(a)

In [ ]:
b = block_DCT.block_analyze(a)

In [ ]:
print(b.astype(np.int16))

In [ ]:
c = block_DCT.block_synthesize(b)

In [ ]:
print(c.astype(np.int16))

## Some routines

In [ ]:
def normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    return 255*(img-min_component)/max_min_component

def print_stats(frame):
    for i in range(frame.shape[2]):
        print("component", i, frame[..., i].max(), frame[..., i].min(), frame[..., i].dtype)

def show_RGB_frame(frame, title=''):
    plt.figure(figsize=(16,16))
    plt.title(title, fontsize=20)
    plt.imshow(cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_BGR2RGB))
    print_stats(frame)

def show_frame(frame, title=''):
    plt.figure(figsize=(16,16))
    plt.title(title, fontsize=20)
    plt.imshow(frame)
    print_stats(frame)

## Testing `block_DCT.analyze()` and `block_DCT.synthesize()`

In [ ]:
x = frame.read(f"../sequences/stockholm/", 0)
#show_frame(normalize(x))
show_RGB_frame(x, title="Original")

In [ ]:
#x = YUV.from_RGB(x.astype(np.int16))
#show_frame(x)

In [ ]:
block_y_side = block_x_side = 8

In [ ]:
y = block_DCT.analyze(x, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(normalize(y), "Block DCT domain")
show_RGB_frame(normalize(y[:64, :64]), "Block DCT domain (detail [0:64, 0:64])")

In [ ]:
z = block_DCT.synthesize(y, block_y_side, block_x_side)

In [ ]:
(x-z).max()

In [ ]:
r = x - z
show_RGB_frame(normalize(r), "Block DCT finite precission error")

In [ ]:
show_RGB_frame(z, "Reconstructed image")

## Testing `block_DCT.constant_quantize()` and `block_DCT.constant_dequantize()`

In [ ]:
block_y_side = block_x_side = 64
q_step = 128
y = block_DCT.analyze(x, block_y_side, block_x_side)
q_y = block_DCT.constant_quantize(y, block_y_side, block_x_side, 128)
dq_y = block_DCT.constant_dequantize(q_y, block_y_side, block_x_side, 128)
z = block_DCT.synthesize(dq_y, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(np.clip(z, a_max=255, a_min=0), "Dequantized (in the block DCT domain) image")

In [ ]:
r = x - z
n = normalize(r)
show_RGB_frame(n, "Quantization error")

## Testing `block_DCT.find_optimal_Q_steps()`

In [ ]:
block_y_side = block_x_side = 64
q_step = 128
y = block_DCT.analyze(x, block_y_side, block_x_side)
slopes, Q_steps = block_DCT.get_slopes(y, block_y_side, block_x_side, q_step)

In [ ]:
print(slopes)

In [ ]:
Q_steps, new_slopes = block_DCT.find_optimal_Q_steps(y, block_y_side, block_x_side, Q_steps, slopes, np.median(slopes))

In [ ]:
print(Q_steps, new_slopes)

In [ ]:
q_y = block_DCT.quantize(y, block_y_side, block_x_side, Q_steps)
dq_y = block_DCT.dequantize(q_y, block_y_side, block_x_side, Q_steps)
z = block_DCT.synthesize(dq_y, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(np.clip(z, a_max=255, a_min=0), "Dequantized (using optimal quantization steps) image")

## Computing the variance of the DCT coefficients

In [ ]:
block_y_side = block_x_side = 64
y = block_DCT.analyze(x, block_y_side, block_x_side)
q_y = block_DCT.constant_quantize(y, block_y_side, block_x_side, 128)
v = block_DCT.compute_variances(q_y, block_y_side, block_x_side)

In [ ]:
print(v)

## Correlating the coefficients (generating subbands)

In [ ]:
block_y_side = block_x_side = 8
Q_step = 32
x = frame.read(f"../sequences/stockholm/", 0)
y = block_DCT.analyze(x, block_y_side, block_x_side)
q_y = block_DCT.constant_quantize(y, block_y_side, block_x_side, Q_step)
dq_y = block_DCT.constant_dequantize(q_y, block_y_side, block_x_side, Q_step)
cor_dq_y = block_DCT.correlate(dq_y, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(dq_y, "Decorrelated (original) dequantized block DCT domain")
show_RGB_frame(cor_dq_y, "Correlated dequantized block DCT domain")
blocks_in_y = x.shape[0]//block_y_side
blocks_in_x = x.shape[1]//block_x_side
show_RGB_frame(normalize(cor_dq_y[:blocks_in_y, :blocks_in_x]))

In [ ]:
decor_dq_y = block_DCT.decorrelate(cor_dq_y, block_y_side, block_x_side)

In [ ]:
show_RGB_frame(decor_dq_y)

In [ ]:
(decor_dq_y == dq_y).all()

## Coding performance

In [ ]:
x = frame.read(f"../sequences/stockholm/", 0)
x = YCoCg.from_RGB(x.astype(np.int16))
